In [1]:
import pandas as pd

In [14]:
datasets = [{"filepath": r"../Resources_cleaned/winemag_full.json",
           "name": "full"},
            {"filepath": r"../Resources_cleaned/winemag_23000.json",
           "name": "23k"},
          {"filepath": r"../Resources_cleaned/winemag_5000.json",
          "name": "5k"},
           ]


In [22]:
df_list = []
for dd in datasets:
    df = pd.read_json(dd["filepath"])
    nb_rows = df.shape[0]
    df_temp = df.groupby("country")['points'].count()/nb_rows
    df_temp = df_temp.rename(dd["name"])
    df_list.append(df_temp)
#     print(df.shape[0])
#     # groupby country and extract the nb of row per country
#     print(df["country"].unique())
    


In [34]:
df_list
merged_dt = pd.concat(df_list, axis=1, sort=False)
# pd.concat([s1, s2], axis=1)
merged_dt = merged_dt.fillna(0)
merged_dt

,full,23k,5k
Argentina,0.035009,0.046851,0.046875
Australia,0.021382,0.064796,0.064844
Chile,0.041160,0.050364,0.050391
China,0.000009,0.000000,0.000000
France,0.165685,0.216861,0.216797
Italy,0.157650,0.253724,0.253711
South Africa,0.012052,0.052353,0.052344
Spain,0.061265,0.195912,0.195898
US,0.505788,0.119138,0.119141


In [35]:
# Sample the database based on the wine production on 2016
# loading the data
prod_file = r"../Resources_raw/wine_production_by_country_2016.csv"
df_prod = pd.read_csv(prod_file,
                      encoding="utf-8",
                      header=None,
                     names=["Country","Year","production"])
df_prod = df_prod.drop(["Year"], axis=1)
# calculating the total production this year
total_production = df_prod["production"].sum()
total_production
# Normalize the production
df_prod["production"] = df_prod["production"]/total_production
# Replace United State by US to match the main dataset labels
df_prod['Country'] = df_prod['Country'].str.replace("United States", "US")

# We will select only the 10 bigest countries
df_prod2 = df_prod[0:9]
# China has only one wine reviewed in the dataset so we just remove it
# df_prod2 = df_prod2.drop(5) 
# df_prod2

df_prod2 = df_prod2.set_index('Country')
df_prod2

,production
Country,
Italy,0.199843
France,0.170789
Spain,0.154299
US,0.093836
Australia,0.051040
China,0.044759
South Africa,0.041225
Chile,0.039654
Argentina,0.036906


In [36]:
merged_dt2 = pd.concat([df_prod2["production"],merged_dt], axis=1, sort=False)
merged_dt2

,production,full,23k,5k
Italy,0.199843,0.157650,0.253724,0.253711
France,0.170789,0.165685,0.216861,0.216797
Spain,0.154299,0.061265,0.195912,0.195898
US,0.093836,0.505788,0.119138,0.119141
Australia,0.051040,0.021382,0.064796,0.064844
China,0.044759,0.000009,0.000000,0.000000
South Africa,0.041225,0.012052,0.052353,0.052344
Chile,0.039654,0.041160,0.050364,0.050391
Argentina,0.036906,0.035009,0.046851,0.046875


In [41]:

merged_dt2['Country'] = merged_dt2.index
merged_dt2

KeyError: 'Level Country must be same as name (None)'

In [ ]:
merged_dt2.to_json("../Resources_cleaned/datasets_vs_production.json",orient='records')